# Exploratory analysis of water quality database

<a id = 'top'><a/>

## TABLE OF CONTENT

## [0. Import datasets](#import_dataset)

### [0.1 Loading datasets](#load_files)

#### [0.1.1 Loading emissions data](#load_emissions)
    
#### [0.1.2 Loading aggregated data](#load_aggregated)

### [0.2 Web scraping](#web_scraping)

## [1. Exploratory analysis of datasets](#exploratory_analysis)

### [1.1 Exploratory analysis of emissions dataset](#explore_emissions)
  
### [1.2 Exploratory analysis of aggregated dataset](#explore_aggregated)
   
## [2. SQL Server connection](#SQL_connection)
    
### [2.1 Create tables in the SQL server database](#create_tables)
    
#### [2.1.1 Create table with emission water quantity data](#table_emissions)
    
#### [2.1.2 Create table with measured water quality data](#table_aggregated)

<a id = 'import_dataset'><a/>
## 0. Import datasets
[Top](#top)
    
[1.](#exploratory_analysis)

<a id = 'load_files'><a/>
### 0.1 Loading datasets
[Top](#top)

In [1]:
from urllib.request import urlopen
import json
import pandas as pd
from pandas import json_normalize
import numpy as np

<a id = 'load_emissions'><a/>
#### 0.1.1 Load emissions data
[Top](#top)

The dataset that gives information about the substances and their amount emitted to the water bodies is referred to as "emissions".

This is imported as json file directly from the website https://discodata.eea.europa.eu/# from the database and table WISE_SOE > latest > Waterbase_T_WISE1_Emissions. The followign query was run on the online server:

SELECT *
FROM [WISE_SOE].[latest].[Waterbase_T_WISE1_Emissions]

This automatically also created a URL leading to the JSON file. However, this by default included only the first 100 records, therefore the URL was modified changin the attribtue "nrOfHits" from 100 to 103285, the total nr of rows of the table.

In [152]:
"""
Import json through URL provided by the database online.
The URL shows by default only 100 records (nrOfHits). Therefore it is necessary to loop through the pages (p).
eea_emission_url = "https://discodata.eea.europa.eu/sql?query=SELECT%20*%0AFROM%20%5BWISE_SOE%5D.%5Blatest%5D.%5BWaterbase_T_WISE1_Emissions%5D&p=1&nrOfHits=103285&mail=null&schema=null"
eea_emission_response = urlopen(eea_emission_url)
emissions_raw = json.loads(eea_emission_response.read())
"""

In [6]:
emissions_raw = {"results":[]}
p = 1
nr = 1000
while True:
    eea_emission_url = "https://discodata.eea.europa.eu/sql?query=SELECT%20*%20FROM%20%5BWISE_SOE%5D.%5Blatest%5D.%5BWaterbase_T_WISE1_Emissions%5D&p={page}&nrOfHits={num_record}&mail=null&schema=null".format(page=p, num_record=nr)
    eea_emission_response = urlopen(eea_emission_url)
    json_data = json.loads(eea_emission_response.read())
    if len(json_data.get("results", []))==0:
        break
    else:
        emissions_raw["results"].extend(json_data.get("results", []))
    p = p + 1

In [7]:
emissions_df_raw = json_normalize(emissions_raw['results'])

In [8]:
emissions_df_raw

,countryCode,spatialUnitIdentifier,spatialUnitIdentifierScheme,phenomenonTimeReferencePeriod,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,parameterEmissionsSourceCategory,parameterEPRTRfacilities,resultEmissionsValue,resultEmissionsUom,procedureEmissionsMethod,resultObservationStatus,Remarks,metadata_versionId,metadata_beginLifeSpanVersion,metadata_statusCode,metadata_observationStatus,metadata_statements,UID
0,AT,AT1000,euRBDCode,2016,CAS_7439-92-1,Lead and its compounds,I,yes,759.500000,kg/a,calculated,X,data derived from EPRTR by ETC,http://discomap.eea.europa.eu/data/wisesoe/der...,2020-06-08 00:00:00.000,experimental,A,None,137076
1,AT,AT1000,euRBDCode,2016,CAS_7439-92-1,Lead and its compounds,U2,yes,280.000000,kg/a,calculated,X,data derived from EPRTR by ETC,http://discomap.eea.europa.eu/data/wisesoe/der...,2020-06-08 00:00:00.000,experimental,A,None,137077
2,AT,AT1000,euRBDCode,2016,CAS_7439-97-6,Mercury and its compounds,I,yes,5.290000,kg/a,calculated,X,data derived from EPRTR by ETC,http://discomap.eea.europa.eu/data/wisesoe/der...,2020-06-08 00:00:00.000,experimental,A,None,137078
3,AT,AT1000,euRBDCode,2016,CAS_7440-02-0,Nickel and its compounds,I,yes,2568.300000,kg/a,calculated,X,data derived from EPRTR by ETC,http://discomap.eea.europa.eu/data/wisesoe/der...,2020-06-08 00:00:00.000,experimental,A,None,137080
4,AT,AT1000,euRBDCode,2016,CAS_7440-02-0,Nickel and its compounds,U2,yes,3690.000000,kg/a,calculated,X,data derived from EPRTR by ETC,http://discomap.eea.europa.eu/data/wisesoe/der...,2020-06-08 00:00:00.000,experimental,A,None,137081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103280,XK,XK,countryCode,2019,EEA_31615-01-7,Total nitrogen,U22,no,9.477000,t/a,measured,A,None,https://cdr.eionet.europa.eu/xk/eea/wise_soe/w...,2021-01-13 09:43:12.000,stable,A,None,162583
103281,XK,XK,countryCode,2020,EEA_31-02-7,Total suspended solids,U22,no,6.959722,t/a,measured,A,None,https://cdr.eionet.europa.eu/xk/eea/wise_soe/w...,2022-01-12 23:18:02.000,stable,A,None,179787
103282,XK,XK,countryCode,2020,EEA_3133-01-5,BOD5,U22,no,3.117451,t/a,measured,A,None,https://cdr.eionet.europa.eu/xk/eea/wise_soe/w...,2022-01-12 23:18:02.000,stable,A,None,179788
103283,XK,XK,countryCode,2020,EEA_3133-03-7,CODCr,U22,no,55.646105,t/a,measured,A,None,https://cdr.eionet.europa.eu/xk/eea/wise_soe/w...,2022-01-12 23:18:02.000,stable,A,None,179789


In [9]:
record, attributes = emissions_df_raw.shape
print(f"The emisisons df is composed of {record} records and {attributes} attributes.")

The emisisons df is composed of 103285 records and 19 attributes.


<a id = 'load_aggregated'><a/>
#### 0.1.2 Loading aggregated data
[Top](#top)

In [ ]:
aggregated_raw = {"results" : []}

p = 1
nr = 1000000

while True:
    aggreagated_url = "https://discodata.eea.europa.eu/sql?query=SELECT%20*%0AFROM%20%5BWISE_SOE%5D.%5Blatest%5D.%5BWaterbase_T_WISE6_AggregatedData%5D&p={page}&nrOfHits={num_record}&mail=null&schema=null".format(page = p, num_record = nr)
    aggregated_response = urlopen(aggreagated_url)
    json_data = json.loads(aggregated_response.read())
    
    if len(json_data.get("results", [])) == 0:
        break
    else:
        aggregated_raw['results'].extend(json_data.get("results", []))
    p = p + 1

# HTTP Error 500 !!

In [ ]:
aggregated_df_raw = json_normlaize(aggregated_raw["results"])

Since it returns an error 500, the following method to import the data can be tried.

The total records number is obtained by counting the total records of the table directly at the user interface query editor of the database website https://discodata.eea.europa.eu/#:
SELECT COUNT(*) AS total_records
FROM [WISE_SOE].[latest].[Waterbase_T_WISE6_AggregatedData]

Afterward, this number is used to substitute the default value (100) of number of records shown in the URL-JSON which would import the Monitoring dataset, obtained with the query SELECT * AS total_records FROM [WISE_SOE].[latest].[Waterbase_T_WISE6_AggregatedData] directly at the user interface query editor of the database website https://discodata.eea.europa.eu/#.

In [18]:
aggreagated_url_tot_rec = "https://discodata.eea.europa.eu/sql?query=SELECT%20COUNT(*)%20AS%20total_records%0AFROM%20%5BWISE_SOE%5D.%5Blatest%5D.%5BWaterbase_T_WISE6_AggregatedData%5D&p=1&nrOfHits=100&mail=null&schema=null"

aggreagated_resp_tot_rec = urlopen(aggreagated_url_tot_rec)
aggregated_tot_rec_raw = json.loads(aggreagated_resp_tot_rec.read())
aggregated_tot_rec_raw2 = json_normalize(aggregated_tot_rec_raw["results"])
aggregated_tot_rec = aggregated_tot_rec_raw2.iloc[0].total_records
aggregated_tot_rec

4550559

In [ ]:
nr_aggr = aggregated_tot_rec

aggreagated_url = "https://discodata.eea.europa.eu/sql?query=SELECT%20*%0AFROM%20%5BWISE_SOE%5D.%5Blatest%5D.%5BWaterbase_T_WISE6_AggregatedData%5D&p=1&nrOfHits={num_rec_aggr}&mail=null&schema=null".format(num_rec_aggr = nr_aggr)
aggregated_response = urlopen(aggreagated_url)
aggregated_raw = json.loads(aggregated_response.read())
aggregated_df_raw = json_normalize(aggregated_raw["results"])
aggregated_df_raw
# Error 500 internal server error

Since also this attempt to access directyl the database returns a 500 error, internal server error, that can't be managed but only by the server admin, the dataset is directly imported as csv file, which has to be first downloaded from the EEA Discodata webite https://discodata.eea.europa.eu/# > WISE_SOE > Waterbase_T_WISE6_AggregatedData.

In [178]:
aggregated_df_raw = pd.read_csv("Aggregated.csv", low_memory=False)
aggregated_df_raw

,countryCode,monitoringSiteIdentifier,monitoringSiteIdentifierScheme,parameterWaterBodyCategory,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,procedureAnalysedMatrix,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,...,procedureAnalyticalMethod,parameterSampleDepth,resultObservationStatus,remarks,metadata_versionId,metadata_beginLifeSpanVersion,metadata_statusCode,metadata_observationStatus,metadata_statements,UID
0,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2004,2004-01--2004-12,...,NaN,-9999.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,1
1,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2005,2005-01--2005-12,...,NaN,-9999.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,2
2,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2006,2006-01--2006-12,...,NaN,-9999.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,3
3,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2007,2007-01--2007-12,...,NaN,-9999.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,4
4,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14797-55-8,Nitrate,W,mg{NO3}/L,2005,2005-01--2005-12,...,NaN,-9999.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4550554,RO,RO85010,euMonitoringSiteCode,RW,CAS_7440-66-6,Zinc and its compounds,W-DIS,ug/L,2017,2017-01-01--2017-12-31,...,Other analytical method,0.0,NaN,EN ISO 8288:2001,http://discomap.eea.europa.eu/data/wisesoe/der...,2019-08-29 07:54:03.000,experimental,A,NaN,17834956
4550555,RO,RO85010,euMonitoringSiteCode,RW,CAS_7440-47-3,Chromium and its compounds,W-DIS,ug/L,2017,2017-01-01--2017-12-31,...,EN ISO 15586:2003,0.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2019-08-29 07:54:03.000,experimental,A,NaN,17834957
4550556,RO,RO85010,euMonitoringSiteCode,RW,CAS_7440-38-2,Arsenic and its compounds,W-DIS,ug/L,2017,2017-01-01--2017-12-31,...,EN ISO 15586:2003,0.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2019-08-29 07:54:03.000,experimental,A,NaN,17834958
4550557,RO,RO85010,euMonitoringSiteCode,RW,EEA_31-02-7,Total suspended solids,W,mg/L,2017,2017-01-01--2017-12-31,...,EN 872:2005,0.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2019-08-29 07:54:03.000,experimental,A,NaN,17834959


In [179]:
records, attributes = aggregated_df_raw.shape
print(f"The aggregated dataset has {records} records and {attributes} attributes.")

The aggregated dataset has 4550559 records and 32 attributes.


<a id = 'web_scraping'><a/>
### 0.2 Web scraping
[Top](#top)

In [ ]:
# Install the required libraries

"""
Run if not installed yet
!pip install BeautifulSoup4
!pip install requests
!pip install pandas
!pip install lxml
"""

In [2]:
# Import the required libraries
from bs4 import BeautifulSoup
import requests
import lxml
import time

### Web scraping populations of European countries

In [56]:
url_pop = "https://www.worldometers.info/population/countries-in-europe-by-population/"
requests.get(url_pop)

<Response [200]>

In [57]:
response_pop = requests.get(url_pop)
bft_pop = BeautifulSoup(response_pop.text, 'lxml')

In [ ]:
bft_pop

In [ ]:
table_pop = bft_pop.find('div', class_ = 'table-responsive')
table_pop

In [60]:
headers_pop = []
for i in table_pop.find_all('th'):
    title = i.text
    headers_pop.append(title)
headers_pop

['#',
 'Country (or dependency)',
 'Population (2023)',
 'Yearly Change',
 'Net Change',
 'Density (P/Km²)',
 'Land Area (Km²)',
 'Migrants (net)',
 'Fert. Rate',
 'Med. Age',
 'Urban Pop %',
 'World Share']

In [61]:
pop_df = pd.DataFrame(columns = headers_pop)
pop_df

,#,Country (or dependency),Population (2023),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share


In [ ]:
for j in table_pop.find_all('tr')[1:]:
    row_data = j.find_all('td')
    print(row_data)

In [63]:
for j in table_pop.find_all('tr')[1:]:
    row_data = j.find_all('td')
    row = [i.text for i in row_data] # Creates lists of values, which correspond to the table's rows
    length = len(pop_df)
    pop_df.loc[length] = row # At the position lenght, assignthe new row of values

In [64]:
pop_df

,#,Country (or dependency),Population (2023),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,1,Russia,"144,444,359",-0.19 %,"-268,955",9,"16,376,870","-136,414",1.5,39,75 %,1.80 %
1,2,Germany,"83,294,633",-0.09 %,"-75,210",239,"348,560","155,751",1.5,45,77 %,1.04 %
2,3,United Kingdom,"67,736,802",0.34 %,"227,866",280,"241,930","165,790",1.6,40,85 %,0.84 %
3,4,France,"64,756,584",0.20 %,"129,956",118,"547,557","67,761",1.8,42,84 %,0.80 %
4,5,Italy,"58,870,762",-0.28 %,"-166,712",200,"294,140","58,496",1.3,48,72 %,0.73 %
5,6,Spain,"47,519,628",-0.08 %,"-39,002",95,"498,800","39,998",1.3,45,80 %,0.59 %
6,7,Ukraine,"36,744,634",-7.45 %,"-2,957,105",63,"579,320","1,784,718",1.3,45,82 %,0.46 %
7,8,Poland,"41,026,067",2.93 %,"1,168,922",134,"306,230","-910,475",1.5,40,55 %,0.51 %
8,9,Romania,"19,892,812",1.19 %,"233,545",86,"230,170","-254,616",1.7,41,53 %,0.25 %
9,10,Netherlands,"17,618,299",0.31 %,"54,285",522,"33,720","29,998",1.6,42,92 %,0.22 %


In [65]:
populations = pop_df.drop(['#', 'Net Change', 'Migrants (net)', 'Fert. Rate', 'World Share'],
                         axis = 1)

In [66]:
populations

,Country (or dependency),Population (2023),Yearly Change,Density (P/Km²),Land Area (Km²),Med. Age,Urban Pop %
0,Russia,"144,444,359",-0.19 %,9,"16,376,870",39,75 %
1,Germany,"83,294,633",-0.09 %,239,"348,560",45,77 %
2,United Kingdom,"67,736,802",0.34 %,280,"241,930",40,85 %
3,France,"64,756,584",0.20 %,118,"547,557",42,84 %
4,Italy,"58,870,762",-0.28 %,200,"294,140",48,72 %
5,Spain,"47,519,628",-0.08 %,95,"498,800",45,80 %
6,Ukraine,"36,744,634",-7.45 %,63,"579,320",45,82 %
7,Poland,"41,026,067",2.93 %,134,"306,230",40,55 %
8,Romania,"19,892,812",1.19 %,86,"230,170",41,53 %
9,Netherlands,"17,618,299",0.31 %,522,"33,720",42,92 %


In [67]:
populations.rename(columns = {'Country (or dependency)':'Country',
                             'Population (2023)':'Population'},
                  inplace = True)

In [68]:
populations['Country'] = populations['Country'].replace(['Czech Republic (Czechia)'], 'Czechia')

In [69]:
populations['Country'] = populations['Country'].replace(['Russia'], 'Russian Federation')

In [70]:
populations['Country'] = populations['Country'].replace(['Moldova'], 'Moldova, Republic of')

In [71]:
countries = ['Austria','Belgium','Bulgaria','Croatia','Cyprus','Czechia','Denmark','Estonia','Finland','France','Germany',
            'Greece','Hungary','Ireland','Italy','Latvia','Lithuania','Luxembourg','Malta','Netherlands','Poland',
             'Portugal','Romania','Slovakia','Slovenia','Spain','Sweden','Iceland','Liechtenstein','Norway',
             'Switzerland','Albania','Montenegro','North Macedonia','Serbia','Turkey','Andorra','Belarus',
             'Bosnia and Herzegovina','Moldova, Republic of','Monaco','Russian Federation','San Marino','Ukraine',
             'United Kingdom','Vatican City']


In [72]:
code = ['AT','BE','BG','HR','CY','CZ','DK','EE','FI','FR','DE','EL','HU','IE','IT','LV','LT','LU','MT','NL','PL','PT',
'RO','SK','SI','ES','SE','IS','LI','NO','CH','AL','ME','MK','RS','TR','AD','BY','BA','MD','MC','RU','SM','UA','UK','VA']

In [73]:
countries_codes = pd.DataFrame(list(zip(countries, code)), columns = ['countries', 'code'])

In [74]:
populations['Code'] = ['na','na','na','na','na','na','na','na','na','na',
                      'na','na','na','na','na','na','na','na','na','na',
                      'na','na','na','na','na','na','na','na','na','na',
                      'na','na','na','na','na','na','na','na','na','na',
                      'na','na','na','na','na','na','na']

In [75]:
for i in range(0, len(populations)):
    for j in range(0, len(countries_codes)):
        if populations['Country'].iloc[i] == countries_codes['countries'].iloc[j]:
            populations['Code'].iloc[i] = countries_codes['code'].iloc[j]
            #country_code.append(countries_codes['code'].iloc[j])
    

In [76]:
populations

,Country,Population,Yearly Change,Density (P/Km²),Land Area (Km²),Med. Age,Urban Pop %,Code
0,Russian Federation,"144,444,359",-0.19 %,9,"16,376,870",39,75 %,RU
1,Germany,"83,294,633",-0.09 %,239,"348,560",45,77 %,DE
2,United Kingdom,"67,736,802",0.34 %,280,"241,930",40,85 %,UK
3,France,"64,756,584",0.20 %,118,"547,557",42,84 %,FR
4,Italy,"58,870,762",-0.28 %,200,"294,140",48,72 %,IT
5,Spain,"47,519,628",-0.08 %,95,"498,800",45,80 %,ES
6,Ukraine,"36,744,634",-7.45 %,63,"579,320",45,82 %,UA
7,Poland,"41,026,067",2.93 %,134,"306,230",40,55 %,PL
8,Romania,"19,892,812",1.19 %,86,"230,170",41,53 %,RO
9,Netherlands,"17,618,299",0.31 %,522,"33,720",42,92 %,NL


In [77]:
countries_codes['Country_present'] = ['na','na','na','na','na','na','na','na','na','na',
                                      'na','na','na','na','na','na','na','na','na','na',
                                      'na','na','na','na','na','na','na','na','na','na',
                                      'na','na','na','na','na','na','na','na','na','na',
                                      'na','na','na','na','na','na']

In [78]:
for i in range(0, len(countries_codes)):
    for j in range(0, len(populations)):
        if countries_codes['countries'].iloc[i] == populations['Country'].iloc[j]:
            countries_codes['Country_present'].iloc[i] = 'Present'

In [79]:
countries_codes

,countries,code,Country_present
0,Austria,AT,Present
1,Belgium,BE,Present
2,Bulgaria,BG,Present
3,Croatia,HR,Present
4,Cyprus,CY,na
5,Czechia,CZ,Present
6,Denmark,DK,Present
7,Estonia,EE,Present
8,Finland,FI,Present
9,France,FR,Present


Missing population values of Vatican City, Turkey and Cyprus.

In [80]:
len(populations)

47

In [81]:
len(countries_codes)

46

In [82]:
populations['Population'] = populations['Population'].str.replace(",", "")

In [83]:
populations['Land Area (Km²)'] = populations['Land Area (Km²)'].str.replace(",", "")

In [84]:
populations['Density (P/Km²)'] = populations['Density (P/Km²)'].str.replace(",", "")

In [85]:
populations['Urban Pop %'] = populations['Urban Pop %'].str.replace(" %", "")

In [86]:
populations['Yearly Change'] = populations['Yearly Change'].str.replace(" %", "")

In [87]:
populations.head()

,Country,Population,Yearly Change,Density (P/Km²),Land Area (Km²),Med. Age,Urban Pop %,Code
0,Russian Federation,144444359,-0.19,9,16376870,39,75,RU
1,Germany,83294633,-0.09,239,348560,45,77,DE
2,United Kingdom,67736802,0.34,280,241930,40,85,UK
3,France,64756584,0.20,118,547557,42,84,FR
4,Italy,58870762,-0.28,200,294140,48,72,IT


In [88]:
turkey_pop = ['Turkey', 85903600, 'null', 112, 769630, 31.8, 77.1, 'TR']
cyprus_pop = ['Cyprus', 920701, 'null', 136, 9213, 38.5, 65.5, 'CY']

In [89]:
populations.loc[len(populations)] = vatican_pop

In [90]:
populations.loc[len(populations)] = turkey_pop

In [91]:
populations.loc[len(populations)] = cyprus_pop

In [92]:
kosovo_pop = ['Kosovo', 1761985, 'null', 166, 10887, 31.5, 50, 'XK']
populations.loc[len(populations)] = kosovo_pop

In [93]:
for i in range(0, len(countries_codes)):
    for j in range(0, len(populations)):
        if countries_codes['countries'].iloc[i] == populations['Country'].iloc[j]:
            countries_codes['Country_present'].iloc[i] = 'Present'
countries_codes

,countries,code,Country_present
0,Austria,AT,Present
1,Belgium,BE,Present
2,Bulgaria,BG,Present
3,Croatia,HR,Present
4,Cyprus,CY,Present
5,Czechia,CZ,Present
6,Denmark,DK,Present
7,Estonia,EE,Present
8,Finland,FI,Present
9,France,FR,Present


Now all the countries have the population value.

In [94]:
populations = populations[~(populations['Country'] == 'Isle of Man')]
populations = populations[~(populations['Country'] == 'Faeroe Islands')]
populations = populations[~(populations['Country'] == 'Gibraltar')]
populations = populations[~(populations['Country'] == 'Holy See')]

In [95]:
populations = populations[~(populations['Country'] == 'Andorra')]
populations = populations[~(populations['Country'] == 'Belarus')]
populations = populations[~(populations['Country'] == 'Belarus')]  
populations = populations[~(populations['Country'] == 'Monaco')]
populations = populations[~(populations['Country'] == 'Moldova, Republic of')]
populations = populations[~(populations['Country'] == 'Russian Federation')]
populations = populations[~(populations['Country'] == 'San Marino')]
populations = populations[~(populations['Country'] == 'Ukraine')]

In [96]:
populations

,Country,Population,Yearly Change,Density (P/Km²),Land Area (Km²),Med. Age,Urban Pop %,Code
1,Germany,83294633,-0.09,239,348560,45,77,DE
2,United Kingdom,67736802,0.34,280,241930,40,85,UK
3,France,64756584,0.20,118,547557,42,84,FR
4,Italy,58870762,-0.28,200,294140,48,72,IT
5,Spain,47519628,-0.08,95,498800,45,80,ES
7,Poland,41026067,2.93,134,306230,40,55,PL
8,Romania,19892812,1.19,86,230170,41,53,RO
9,Netherlands,17618299,0.31,522,33720,42,92,NL
10,Belgium,11686140,0.26,386,30280,41,99,BE
11,Greece,10341277,-0.42,80,128900,45,86,EL


In [97]:
populations.to_csv("Populations.csv")

<a id = 'exploratory_analysis'></a>
## 1. Exploratory analysis of the datasets
[Top](#top)

[0.](#import_dataset)


<a id = 'explore_emissions'></a>
### 1.1 Exploratory analysis of emissions dataset
[Top](#top)

In [71]:
emissions_df_raw.columns

Index(['countryCode', 'spatialUnitIdentifier', 'spatialUnitIdentifierScheme',
       'phenomenonTimeReferencePeriod', 'observedPropertyDeterminandCode',
       'observedPropertyDeterminandLabel', 'parameterEmissionsSourceCategory',
       'parameterEPRTRfacilities', 'resultEmissionsValue',
       'resultEmissionsUom', 'procedureEmissionsMethod',
       'resultObservationStatus', 'Remarks', 'metadata_versionId',
       'metadata_beginLifeSpanVersion', 'metadata_statusCode',
       'metadata_observationStatus', 'metadata_statements', 'UID'],
      dtype='object')

#### Null values

In [72]:
emissions_df_raw.isnull().sum().sum()

299218

In [73]:
emissions_df_raw[emissions_df_raw.isnull().any(axis=1)]

,countryCode,spatialUnitIdentifier,spatialUnitIdentifierScheme,phenomenonTimeReferencePeriod,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,parameterEmissionsSourceCategory,parameterEPRTRfacilities,resultEmissionsValue,resultEmissionsUom,procedureEmissionsMethod,resultObservationStatus,Remarks,metadata_versionId,metadata_beginLifeSpanVersion,metadata_statusCode,metadata_observationStatus,metadata_statements,UID
0,AT,AT1000,euRBDCode,2016,CAS_7439-92-1,Lead and its compounds,I,yes,759.500000,kg/a,calculated,X,data derived from EPRTR by ETC,http://discomap.eea.europa.eu/data/wisesoe/der...,2020-06-08 00:00:00.000,experimental,A,None,137076
1,AT,AT1000,euRBDCode,2016,CAS_7439-92-1,Lead and its compounds,U2,yes,280.000000,kg/a,calculated,X,data derived from EPRTR by ETC,http://discomap.eea.europa.eu/data/wisesoe/der...,2020-06-08 00:00:00.000,experimental,A,None,137077
2,AT,AT1000,euRBDCode,2016,CAS_7439-97-6,Mercury and its compounds,I,yes,5.290000,kg/a,calculated,X,data derived from EPRTR by ETC,http://discomap.eea.europa.eu/data/wisesoe/der...,2020-06-08 00:00:00.000,experimental,A,None,137078
3,AT,AT1000,euRBDCode,2016,CAS_7440-02-0,Nickel and its compounds,I,yes,2568.300000,kg/a,calculated,X,data derived from EPRTR by ETC,http://discomap.eea.europa.eu/data/wisesoe/der...,2020-06-08 00:00:00.000,experimental,A,None,137080
4,AT,AT1000,euRBDCode,2016,CAS_7440-02-0,Nickel and its compounds,U2,yes,3690.000000,kg/a,calculated,X,data derived from EPRTR by ETC,http://discomap.eea.europa.eu/data/wisesoe/der...,2020-06-08 00:00:00.000,experimental,A,None,137081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103280,XK,XK,countryCode,2019,EEA_31615-01-7,Total nitrogen,U22,no,9.477000,t/a,measured,A,None,https://cdr.eionet.europa.eu/xk/eea/wise_soe/w...,2021-01-13 09:43:12.000,stable,A,None,162583
103281,XK,XK,countryCode,2020,EEA_31-02-7,Total suspended solids,U22,no,6.959722,t/a,measured,A,None,https://cdr.eionet.europa.eu/xk/eea/wise_soe/w...,2022-01-12 23:18:02.000,stable,A,None,179787
103282,XK,XK,countryCode,2020,EEA_3133-01-5,BOD5,U22,no,3.117451,t/a,measured,A,None,https://cdr.eionet.europa.eu/xk/eea/wise_soe/w...,2022-01-12 23:18:02.000,stable,A,None,179788
103283,XK,XK,countryCode,2020,EEA_3133-03-7,CODCr,U22,no,55.646105,t/a,measured,A,None,https://cdr.eionet.europa.eu/xk/eea/wise_soe/w...,2022-01-12 23:18:02.000,stable,A,None,179789


In [74]:
emissions_df_raw = emissions_df_raw.drop(['metadata_versionId',
                                           'metadata_beginLifeSpanVersion', 'metadata_statusCode',
                                           'metadata_observationStatus', 'metadata_statements'], axis = 1)

In [75]:
emissions_df_raw[emissions_df_raw.isnull().any(axis=1)]

,countryCode,spatialUnitIdentifier,spatialUnitIdentifierScheme,phenomenonTimeReferencePeriod,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,parameterEmissionsSourceCategory,parameterEPRTRfacilities,resultEmissionsValue,resultEmissionsUom,procedureEmissionsMethod,resultObservationStatus,Remarks,UID
107,AT,AT1001,euSubUnitCode,2004--2007,CAS_7723-14-0,Total phosphorus,NP,None,4722.000000,t/a,None,None,None,169087
108,AT,AT1001,euSubUnitCode,2004--2007,CAS_7723-14-0,Total phosphorus,NP1,None,1529.000000,t/a,None,None,None,169088
109,AT,AT1001,euSubUnitCode,2004--2007,CAS_7723-14-0,Total phosphorus,NP2,None,44.000000,t/a,None,None,None,169089
110,AT,AT1001,euSubUnitCode,2004--2007,CAS_7723-14-0,Total phosphorus,NP4,None,299.000000,t/a,None,None,None,169090
111,AT,AT1001,euSubUnitCode,2004--2007,EEA_31615-01-7,Total nitrogen,NP,None,83218.000000,t/a,None,None,None,169091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103280,XK,XK,countryCode,2019,EEA_31615-01-7,Total nitrogen,U22,no,9.477000,t/a,measured,A,None,162583
103281,XK,XK,countryCode,2020,EEA_31-02-7,Total suspended solids,U22,no,6.959722,t/a,measured,A,None,179787
103282,XK,XK,countryCode,2020,EEA_3133-01-5,BOD5,U22,no,3.117451,t/a,measured,A,None,179788
103283,XK,XK,countryCode,2020,EEA_3133-03-7,CODCr,U22,no,55.646105,t/a,measured,A,None,179789


In [76]:
emissions_df_raw['countryCode'].unique()
emissions_df_raw['spatialUnitIdentifier'].unique()
emissions_df_raw['observedPropertyDeterminandCode'].unique()
emissions_df_raw['resultEmissionsValue'].unique()
emissions_df_raw['resultEmissionsUom'].unique()
emissions_df_raw['parameterEmissionsSourceCategory'].unique()
emissions_df_raw['spatialUnitIdentifierScheme'].unique()

array(['euRBDCode', 'euSubUnitCode', 'eionetSubUnitCode', 'countryCode',
       'eionetRBDCode'], dtype=object)

In [77]:
emissions_df_raw = emissions_df_raw.dropna(subset = ['observedPropertyDeterminandCode', 'resultEmissionsValue', 'resultEmissionsUom'])

In [78]:
emissions_df_raw.isnull().sum().sum()

186743

In [79]:
emissions_df_raw[emissions_df_raw['resultEmissionsUom'] == "None"]
emissions_df_raw[emissions_df_raw['resultEmissionsUom'].isnull()]
emissions_df_raw['resultEmissionsUom'].isnull().sum()

0

#### Normalizing unit of measure columns

In [80]:
# Normalizing measured data from kg/a to t/a to have comparable data
emissions_df_raw['resultsEmissionsValueNEW'] = np.where(emissions_df_raw['resultEmissionsUom'] == "kg/a",
                                                       emissions_df_raw['resultEmissionsValue']/1000,
                                                        emissions_df_raw['resultEmissionsValue'])
emissions_df_raw['resultEmissionsUomNEW'] = np.where(emissions_df_raw['resultEmissionsUom'] == "kg/a",
                                                     "t/a",
                                                     emissions_df_raw['resultEmissionsUom'])
emissions_df_raw

,countryCode,spatialUnitIdentifier,spatialUnitIdentifierScheme,phenomenonTimeReferencePeriod,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,parameterEmissionsSourceCategory,parameterEPRTRfacilities,resultEmissionsValue,resultEmissionsUom,procedureEmissionsMethod,resultObservationStatus,Remarks,UID,resultsEmissionsValueNEW,resultEmissionsUomNEW
0,AT,AT1000,euRBDCode,2016,CAS_7439-92-1,Lead and its compounds,I,yes,759.500000,kg/a,calculated,X,data derived from EPRTR by ETC,137076,0.759500,t/a
1,AT,AT1000,euRBDCode,2016,CAS_7439-92-1,Lead and its compounds,U2,yes,280.000000,kg/a,calculated,X,data derived from EPRTR by ETC,137077,0.280000,t/a
2,AT,AT1000,euRBDCode,2016,CAS_7439-97-6,Mercury and its compounds,I,yes,5.290000,kg/a,calculated,X,data derived from EPRTR by ETC,137078,0.005290,t/a
3,AT,AT1000,euRBDCode,2016,CAS_7440-02-0,Nickel and its compounds,I,yes,2568.300000,kg/a,calculated,X,data derived from EPRTR by ETC,137080,2.568300,t/a
4,AT,AT1000,euRBDCode,2016,CAS_7440-02-0,Nickel and its compounds,U2,yes,3690.000000,kg/a,calculated,X,data derived from EPRTR by ETC,137081,3.690000,t/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103280,XK,XK,countryCode,2019,EEA_31615-01-7,Total nitrogen,U22,no,9.477000,t/a,measured,A,None,162583,9.477000,t/a
103281,XK,XK,countryCode,2020,EEA_31-02-7,Total suspended solids,U22,no,6.959722,t/a,measured,A,None,179787,6.959722,t/a
103282,XK,XK,countryCode,2020,EEA_3133-01-5,BOD5,U22,no,3.117451,t/a,measured,A,None,179788,3.117451,t/a
103283,XK,XK,countryCode,2020,EEA_3133-03-7,CODCr,U22,no,55.646105,t/a,measured,A,None,179789,55.646105,t/a


In [81]:
emissions_df_raw = emissions_df_raw.drop(['resultEmissionsUom', 'resultEmissionsValue'], axis = 1)

In [82]:
emissions_df_raw

,countryCode,spatialUnitIdentifier,spatialUnitIdentifierScheme,phenomenonTimeReferencePeriod,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,parameterEmissionsSourceCategory,parameterEPRTRfacilities,procedureEmissionsMethod,resultObservationStatus,Remarks,UID,resultsEmissionsValueNEW,resultEmissionsUomNEW
0,AT,AT1000,euRBDCode,2016,CAS_7439-92-1,Lead and its compounds,I,yes,calculated,X,data derived from EPRTR by ETC,137076,0.759500,t/a
1,AT,AT1000,euRBDCode,2016,CAS_7439-92-1,Lead and its compounds,U2,yes,calculated,X,data derived from EPRTR by ETC,137077,0.280000,t/a
2,AT,AT1000,euRBDCode,2016,CAS_7439-97-6,Mercury and its compounds,I,yes,calculated,X,data derived from EPRTR by ETC,137078,0.005290,t/a
3,AT,AT1000,euRBDCode,2016,CAS_7440-02-0,Nickel and its compounds,I,yes,calculated,X,data derived from EPRTR by ETC,137080,2.568300,t/a
4,AT,AT1000,euRBDCode,2016,CAS_7440-02-0,Nickel and its compounds,U2,yes,calculated,X,data derived from EPRTR by ETC,137081,3.690000,t/a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103280,XK,XK,countryCode,2019,EEA_31615-01-7,Total nitrogen,U22,no,measured,A,None,162583,9.477000,t/a
103281,XK,XK,countryCode,2020,EEA_31-02-7,Total suspended solids,U22,no,measured,A,None,179787,6.959722,t/a
103282,XK,XK,countryCode,2020,EEA_3133-01-5,BOD5,U22,no,measured,A,None,179788,3.117451,t/a
103283,XK,XK,countryCode,2020,EEA_3133-03-7,CODCr,U22,no,measured,A,None,179789,55.646105,t/a


#### Split date columns

In [86]:
emissions_df_raw['phenomenonTimeReferencePeriod'].unique()

array(['2016', '2017', '2018', '2019', '2020', '2004--2007', '2007',
       '2009--2011', '2010', '2011', '2012', '2014', '2015', '2005',
       '1998', '2000', '2001', '2002', '2003', '2004', '2006', '2008',
       '2009', '2013', '1977--1998', '1987--1998', '1985', '1992', '1995',
       '1996', '2009--2014', '2012--2015', '2011--2014', '2000--2006',
       '2009--2012', '2016--2018', '2018--2019', '2019--2020',
       '2013--2014'], dtype=object)

In [87]:
emissions_df_raw[['TimeReferenceStart', 'TimeReferenceEnd']] = emissions_df_raw['phenomenonTimeReferencePeriod'].str.split('--', expand = True)

In [88]:
emissions_df_raw['TimeReferenceEnd'].isna().sum()

97366

In [89]:
emissions_df_raw['TimeReferenceEnd'] = np.where(emissions_df_raw['TimeReferenceEnd'].isnull(),
                                                emissions_df_raw['TimeReferenceStart'],
                                                emissions_df_raw['TimeReferenceEnd'])

In [90]:
emissions_df_raw = emissions_df_raw.drop(['phenomenonTimeReferencePeriod'], axis = 1)

#### Duplicates

In [91]:
emissions_df_raw[emissions_df_raw.duplicated(keep=False)]

,countryCode,spatialUnitIdentifier,spatialUnitIdentifierScheme,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,parameterEmissionsSourceCategory,parameterEPRTRfacilities,procedureEmissionsMethod,resultObservationStatus,Remarks,UID,resultsEmissionsValueNEW,resultEmissionsUomNEW,TimeReferenceStart,TimeReferenceEnd


There are no duplicated rows.

In [92]:
emissions_df_raw

,countryCode,spatialUnitIdentifier,spatialUnitIdentifierScheme,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,parameterEmissionsSourceCategory,parameterEPRTRfacilities,procedureEmissionsMethod,resultObservationStatus,Remarks,UID,resultsEmissionsValueNEW,resultEmissionsUomNEW,TimeReferenceStart,TimeReferenceEnd
0,AT,AT1000,euRBDCode,CAS_7439-92-1,Lead and its compounds,I,yes,calculated,X,data derived from EPRTR by ETC,137076,0.759500,t/a,2016,2016
1,AT,AT1000,euRBDCode,CAS_7439-92-1,Lead and its compounds,U2,yes,calculated,X,data derived from EPRTR by ETC,137077,0.280000,t/a,2016,2016
2,AT,AT1000,euRBDCode,CAS_7439-97-6,Mercury and its compounds,I,yes,calculated,X,data derived from EPRTR by ETC,137078,0.005290,t/a,2016,2016
3,AT,AT1000,euRBDCode,CAS_7440-02-0,Nickel and its compounds,I,yes,calculated,X,data derived from EPRTR by ETC,137080,2.568300,t/a,2016,2016
4,AT,AT1000,euRBDCode,CAS_7440-02-0,Nickel and its compounds,U2,yes,calculated,X,data derived from EPRTR by ETC,137081,3.690000,t/a,2016,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103280,XK,XK,countryCode,EEA_31615-01-7,Total nitrogen,U22,no,measured,A,None,162583,9.477000,t/a,2019,2019
103281,XK,XK,countryCode,EEA_31-02-7,Total suspended solids,U22,no,measured,A,None,179787,6.959722,t/a,2020,2020
103282,XK,XK,countryCode,EEA_3133-01-5,BOD5,U22,no,measured,A,None,179788,3.117451,t/a,2020,2020
103283,XK,XK,countryCode,EEA_3133-03-7,CODCr,U22,no,measured,A,None,179789,55.646105,t/a,2020,2020


#### Emissions source categories

In [93]:
emissions_df_raw['observedPropertyDeterminandLabel'].unique()
emissions_df_raw['parameterEmissionsSourceCategory'].unique()

array(['I', 'U2', 'NP', 'NP1', 'NP2', 'NP4', 'U21', 'U22', 'U23', 'U24',
       'I4', 'NP3', 'NP7', 'U', 'U1', 'NP72', 'O', 'NP8', 'I3', 'NP5',
       'PT', 'O2', 'O3', 'O4', 'U11', 'NP71', 'U12', 'U13', 'O1', 'U14'],
      dtype=object)

In [94]:
emission_source_category = ['PT', 
                            'U',
                            'U1', 'U11', 'U12', 'U13', 'U14',
                            'U2', 'U21', 'U22', 'U23', 'U24',
                            'I', 'I3', 'I4', 
                            'O', 'O1', 'O2', 'O3', 'O4', 
                            'NP',
                            'NP1', 'NP2', 'NP3', 'NP4', 'NP5', 'NP7', 'NP8',
                            'NP71', 'NP72', 'NP73', 'NP74']
emission_source_category_label = ['Point Sources',
                                  'Point Urban Wastewater',
                                  'Point Urban Wastewater Untreated',
                                  'Point Urban Wastewater Untreated less than 2000 p.e.',
                                  'Point Urban Wastewater Untreated between 2000 and 10000 p.e.',
                                  'Point Urban Wastewater Untreated between 10000 and 100000 p.e.',
                                  'Point Urban Wastewater Untreated more than 100000 p.e.',
                                  'Point Urban Wastewater Treated',
                                  'Point Urban Wastewater Treated less than 2000 p.e.',
                                  'Point Urban Wastewater Treated between 2000 and 10000 p.e.',
                                  'Point Urban Wastewater Treated between 10000 and 100000 p.e.',
                                  'Point Urban Wastewater Treated more than 100000 p.e.',
                                  'Point Industrial Wastewater',
                                  'Point Industrial Wastewater Treated',
                                  'Point Industrial Wastewater Untreated',
                                  'Point Other point emissions',
                                  'Point Contaminated sites or abandoned industrial sites',
                                  'Point Waste disposal sites',
                                  'Point Mine waters',
                                  'Point Aquaculture',
                                  'Diffuse sources',
                                  'Diffuse Agricultural emissions',
                                  'Diffuse Atmospheric deposition',
                                  'Diffuse Un-connected dwellings emissions',
                                  'Diffuse Urban run-off',
                                  'Diffuse Storm overflow emissions',
                                  'Diffuse Other diffuse emissions',
                                  'Diffuse Background emissions',
                                  'Diffuse Other Forestry emissions',
                                  'Diffuse Other Transport emissions',
                                  'Diffuse Other Mining emissions',
                                  'Diffuse Other Aquaculture emissions']

In [95]:
emission_category = pd.DataFrame(list(zip(emission_source_category, emission_source_category_label)), columns = ['EmissionSourceCat', 'EmissionSourceCatLabel'])
emission_category

,EmissionSourceCat,EmissionSourceCatLabel
0,PT,Point Sources
1,U,Point Urban Wastewater
2,U1,Point Urban Wastewater Untreated
3,U11,Point Urban Wastewater Untreated less than 200...
4,U12,Point Urban Wastewater Untreated between 2000 ...
5,U13,Point Urban Wastewater Untreated between 10000...
6,U14,Point Urban Wastewater Untreated more than 100...
7,U2,Point Urban Wastewater Treated
8,U21,Point Urban Wastewater Treated less than 2000 ...
9,U22,Point Urban Wastewater Treated between 2000 an...


In [96]:
emissions_df_raw.columns

Index(['countryCode', 'spatialUnitIdentifier', 'spatialUnitIdentifierScheme',
       'observedPropertyDeterminandCode', 'observedPropertyDeterminandLabel',
       'parameterEmissionsSourceCategory', 'parameterEPRTRfacilities',
       'procedureEmissionsMethod', 'resultObservationStatus', 'Remarks', 'UID',
       'resultsEmissionsValueNEW', 'resultEmissionsUomNEW',
       'TimeReferenceStart', 'TimeReferenceEnd'],
      dtype='object')

In [97]:
emissions_df_raw.shape

(98045, 15)

#### Save cleaned df

In [98]:
emission_category.to_csv("emission_category.csv")

In [99]:
emissions_df_raw.to_csv("emissions_cleaned.csv")

<a id = 'explore_aggregated'></a>
### 1.2 Exploratory analysis of aggregated dataset
[Top](#top)

In [134]:
aggregated_df_raw.columns

Index(['countryCode', 'monitoringSiteIdentifier',
       'monitoringSiteIdentifierScheme', 'parameterWaterBodyCategory',
       'observedPropertyDeterminandCode', 'observedPropertyDeterminandLabel',
       'procedureAnalysedMatrix', 'resultUom', 'phenomenonTimeReferenceYear',
       'parameterSamplingPeriod', 'procedureLOQValue', 'resultNumberOfSamples',
       'resultQualityNumberOfSamplesBelowLOQ', 'resultQualityMinimumBelowLOQ',
       'resultMinimumValue', 'resultQualityMeanBelowLOQ', 'resultMeanValue',
       'resultQualityMaximumBelowLOQ', 'resultMaximumValue',
       'resultQualityMedianBelowLOQ', 'resultMedianValue',
       'resultStandardDeviationValue', 'procedureAnalyticalMethod',
       'parameterSampleDepth', 'resultObservationStatus', 'remarks',
       'metadata_versionId', 'metadata_beginLifeSpanVersion',
       'metadata_statusCode', 'metadata_observationStatus',
       'metadata_statements', 'UID'],
      dtype='object')

In [137]:
aggregated_df_raw.head()

,countryCode,monitoringSiteIdentifier,monitoringSiteIdentifierScheme,parameterWaterBodyCategory,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,procedureAnalysedMatrix,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,...,procedureAnalyticalMethod,parameterSampleDepth,resultObservationStatus,remarks,metadata_versionId,metadata_beginLifeSpanVersion,metadata_statusCode,metadata_observationStatus,metadata_statements,UID
0,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2004,2004-01--2004-12,...,NaN,-9999.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,1
1,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2005,2005-01--2005-12,...,NaN,-9999.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,2
2,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2006,2006-01--2006-12,...,NaN,-9999.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,3
3,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2007,2007-01--2007-12,...,NaN,-9999.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,4
4,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14797-55-8,Nitrate,W,mg{NO3}/L,2005,2005-01--2005-12,...,NaN,-9999.0,NaN,NaN,http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,5


In [138]:
aggregated_df_raw['resultObservationStatus'].unique() # nan, 'A', 'O'
aggregated_df_raw['metadata_statusCode'].unique() # 'experimental', 'valid', 'stable'
aggregated_df_raw['metadata_observationStatus'].unique() # 'A', 'U'
aggregated_df_raw['metadata_statements'].unique() # 
aggregated_df_raw['remarks'].unique() # 
aggregated_df_raw['procedureAnalyticalMethod'].unique() #

array([nan, 'ISO 7890-3 : 2000', 'EN 26777:1993', ...,
       'APAT CNR IRSA 9020 Man 29 2005',
       'APAT CNR IRSA 4110 A2 Man 29 2015',
       'APAT CNR IRSA 4110 A2 Man 29 2006'], dtype=object)

In [139]:
aggregated_df_raw[~(aggregated_df_raw['remarks'].isna())]

,countryCode,monitoringSiteIdentifier,monitoringSiteIdentifierScheme,parameterWaterBodyCategory,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,procedureAnalysedMatrix,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,...,procedureAnalyticalMethod,parameterSampleDepth,resultObservationStatus,remarks,metadata_versionId,metadata_beginLifeSpanVersion,metadata_statusCode,metadata_observationStatus,metadata_statements,UID
168,BA,BAB3,eionetMonitoringSiteCode,LW,CAS_7439-89-6,Iron and its compounds,W,ug/L,2005,2005-01--2005-12,...,NaN,11.17,NaN,"0-1m,1-7m,7-bottom,0-2m,19-bottom,10-bottom,0-...",http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,3755
170,BA,BAB3,eionetMonitoringSiteCode,LW,CAS_7439-96-5,Manganese and its compounds,W,ug/L,2005,2005-01--2005-12,...,NaN,11.17,NaN,"0-1m,1-7m,7-bottom,0-2m,19-bottom,10-bottom,0-...",http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,3757
172,BA,BAB3,eionetMonitoringSiteCode,LW,CAS_7439-97-6,Mercury and its compounds,W,ug/L,2005,2005-01--2005-12,...,NaN,12.06,NaN,"0-1m,7-bottom,0-2m,19-bottom,10-bottom,0-4m,17...",http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,3759
173,BA,BAB3,eionetMonitoringSiteCode,LW,CAS_7439-97-6,Mercury and its compounds,W,ug/L,2006,2006-01--2006-12,...,NaN,9.56,NaN,"0-2,2-bottom,0-3,3-bottom,0-4,4-8,8-bottom",http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,3760
174,BA,BAB3,eionetMonitoringSiteCode,LW,CAS_7440-02-0,Nickel and its compounds,W,ug/L,2005,2005-01--2005-12,...,NaN,11.17,NaN,"0-1m,1-7m,7-bottom,0-2m,19-bottom,10-bottom,0-...",http://discomap.eea.europa.eu/data/wisesoe/der...,2015-11-30 00:00:00.000,experimental,A,NaN,3761
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4550544,RO,RO144300_3,euMonitoringSiteCode,RW,CAS_7439-97-6,Mercury and its compounds,W-DIS,ug/L,2017,2017-01-01--2017-12-31,...,Other analytical method,0.00,NaN,EN ISO 17852:2006,http://discomap.eea.europa.eu/data/wisesoe/der...,2019-08-29 07:54:03.000,experimental,A,NaN,17834946
4550548,RO,RO85010,euMonitoringSiteCode,RW,CAS_14798-03-9,Ammonium,W,mg{NH4}/L,2017,2017-01-01--2017-12-31,...,Other analytical method,0.00,NaN,ISO 7150-1:2001,http://discomap.eea.europa.eu/data/wisesoe/der...,2019-08-29 07:54:03.000,experimental,A,NaN,17834950
4550549,RO,RO85010,euMonitoringSiteCode,RW,CAS_14797-55-8,Nitrate,W,mg{NO3}/L,2017,2017-01-01--2017-12-31,...,Other analytical method,0.00,NaN,ISO 7890-3:2000,http://discomap.eea.europa.eu/data/wisesoe/der...,2019-08-29 07:54:03.000,experimental,A,NaN,17834951
4550554,RO,RO85010,euMonitoringSiteCode,RW,CAS_7440-66-6,Zinc and its compounds,W-DIS,ug/L,2017,2017-01-01--2017-12-31,...,Other analytical method,0.00,NaN,EN ISO 8288:2001,http://discomap.eea.europa.eu/data/wisesoe/der...,2019-08-29 07:54:03.000,experimental,A,NaN,17834956


In [140]:
aggregated = aggregated_df_raw.drop(['metadata_versionId', 'metadata_beginLifeSpanVersion', 'remarks',
                                     'procedureAnalyticalMethod', 'metadata_statusCode', 'metadata_observationStatus'],
                                      axis = 1)

In [141]:
aggregated = aggregated[aggregated['metadata_statements'].isna()]

In [142]:
aggregated['metadata_statements'].unique()

array([nan], dtype=object)

In [143]:
aggregated = aggregated.drop(['metadata_statements'],
                            axis = 1)

In [144]:
aggregated[aggregated['resultObservationStatus'] == 'O'] # The results are actually there

,countryCode,monitoringSiteIdentifier,monitoringSiteIdentifierScheme,parameterWaterBodyCategory,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,procedureAnalysedMatrix,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,...,resultQualityMeanBelowLOQ,resultMeanValue,resultQualityMaximumBelowLOQ,resultMaximumValue,resultQualityMedianBelowLOQ,resultMedianValue,resultStandardDeviationValue,parameterSampleDepth,resultObservationStatus,UID
2610347,SK,SKIDK002,euMonitoringSiteCode,RW,EEA_3133-05-9,Dissolved organic carbon (DOC),W-DIS,mg{C}/L,2013,2013-02-05--2013-12-03,...,False,4.157500,False,4.96,False,4.155,0.493206,0.25,O,11196309
2610348,SK,SKIDK002,euMonitoringSiteCode,RW,EEA_3133-05-9,Dissolved organic carbon (DOC),W-DIS,mg{C}/L,2014,2014-02-12--2014-11-26,...,False,4.235833,False,5.10,False,4.350,0.448506,0.25,O,11196310
2610349,SK,SKIDK003,euMonitoringSiteCode,RW,EEA_3133-05-9,Dissolved organic carbon (DOC),W-DIS,mg{C}/L,2013,2013-01-22--2013-12-03,...,False,4.420833,False,5.98,False,4.280,0.831519,0.25,O,11196311
2610350,SK,SKIDK003,euMonitoringSiteCode,RW,EEA_3133-05-9,Dissolved organic carbon (DOC),W-DIS,mg{C}/L,2014,2014-01-21--2014-11-11,...,False,4.420000,False,5.70,False,4.300,0.500511,0.25,O,11196312
2610351,SK,SKIDK005,euMonitoringSiteCode,RW,EEA_3133-05-9,Dissolved organic carbon (DOC),W-DIS,mg{C}/L,2013,2013-01-22--2013-12-03,...,False,4.766667,False,5.70,False,4.700,0.516935,0.25,O,11196313
2610352,SK,SKIDK005,euMonitoringSiteCode,RW,EEA_3133-05-9,Dissolved organic carbon (DOC),W-DIS,mg{C}/L,2014,2014-01-21--2014-11-11,...,False,4.957143,False,6.30,False,5.000,0.698687,0.25,O,11196314


#### Null values

In [145]:
aggregated.isnull().sum()

countryCode                                   0
monitoringSiteIdentifier                      0
monitoringSiteIdentifierScheme                0
parameterWaterBodyCategory                    0
observedPropertyDeterminandCode               0
observedPropertyDeterminandLabel              0
procedureAnalysedMatrix                       0
resultUom                                     0
phenomenonTimeReferenceYear                   0
parameterSamplingPeriod                   69786
procedureLOQValue                       1437394
resultNumberOfSamples                     62926
resultQualityNumberOfSamplesBelowLOQ    1771224
resultQualityMinimumBelowLOQ            1366688
resultMinimumValue                        98666
resultQualityMeanBelowLOQ               1364321
resultMeanValue                              47
resultQualityMaximumBelowLOQ            1365736
resultMaximumValue                        97182
resultQualityMedianBelowLOQ             2020003
resultMedianValue                       

In [146]:
aggregated[aggregated['resultMeanValue'].isna()] # Also the values in the column Median are null. Therefore those rows will be dropped.

,countryCode,monitoringSiteIdentifier,monitoringSiteIdentifierScheme,parameterWaterBodyCategory,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,procedureAnalysedMatrix,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,...,resultQualityMeanBelowLOQ,resultMeanValue,resultQualityMaximumBelowLOQ,resultMaximumValue,resultQualityMedianBelowLOQ,resultMedianValue,resultStandardDeviationValue,parameterSampleDepth,resultObservationStatus,UID
1080603,IT,IT12-4_30,eionetMonitoringSiteCode,LW,CAS_12002-48-1,Trichlorobenzenes (all isomers),W,ug/L,2010,2010-01--2010-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,9257422
2096957,IT,IT12L3_44,euMonitoringSiteCode,LW,CAS_12002-48-1,Trichlorobenzenes (all isomers),W,ug/L,2010,2010-01--2010-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,10624804
2096958,IT,IT12L3_44,euMonitoringSiteCode,LW,CAS_12002-48-1,Trichlorobenzenes (all isomers),W,ug/L,2010,2010-01--2010-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.0,NaN,10624805
2097039,IT,IT12L3_44,euMonitoringSiteCode,LW,CAS_12002-48-1,Trichlorobenzenes (all isomers),W,ug/L,2010,2010-01--2010-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,10624886
2097053,IT,IT12L3_42,euMonitoringSiteCode,LW,CAS_12002-48-1,Trichlorobenzenes (all isomers),W,ug/L,2010,2010-01--2010-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30.0,NaN,10624900
2097060,IT,IT12L3_57,euMonitoringSiteCode,LW,CAS_12002-48-1,Trichlorobenzenes (all isomers),W,ug/L,2010,2010-01--2010-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.3,NaN,10624907
2097068,IT,IT12L3_44,euMonitoringSiteCode,LW,CAS_12002-48-1,Trichlorobenzenes (all isomers),W,ug/L,2010,2010-01--2010-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,10624915
2097099,IT,IT12L4_26,euMonitoringSiteCode,LW,CAS_56-23-5,Carbon tetrachloride,W,ug/L,2010,2010-01--2010-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,10624946
2097118,IT,IT12L4_26,euMonitoringSiteCode,LW,CAS_56-23-5,Carbon tetrachloride,W,ug/L,2010,2010-01--2010-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50.0,NaN,10624965
2097164,IT,IT12L4_26,euMonitoringSiteCode,LW,CAS_12002-48-1,Trichlorobenzenes (all isomers),W,ug/L,2010,2010-01--2010-12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.2,NaN,10625011


In [147]:
aggregated = aggregated.dropna(subset = ['resultMeanValue'], axis = 0)

In [148]:
aggregated['parameterSamplingPeriod'] = np.where(aggregated['parameterSamplingPeriod'].isna(),
                                                aggregated['phenomenonTimeReferenceYear'],
                                                aggregated['parameterSamplingPeriod'])

In [149]:
aggregated.isnull().sum()

countryCode                                   0
monitoringSiteIdentifier                      0
monitoringSiteIdentifierScheme                0
parameterWaterBodyCategory                    0
observedPropertyDeterminandCode               0
observedPropertyDeterminandLabel              0
procedureAnalysedMatrix                       0
resultUom                                     0
phenomenonTimeReferenceYear                   0
parameterSamplingPeriod                       0
procedureLOQValue                       1437351
resultNumberOfSamples                     62883
resultQualityNumberOfSamplesBelowLOQ    1771177
resultQualityMinimumBelowLOQ            1366641
resultMinimumValue                        98619
resultQualityMeanBelowLOQ               1364274
resultMeanValue                               0
resultQualityMaximumBelowLOQ            1365689
resultMaximumValue                        97135
resultQualityMedianBelowLOQ             2019956
resultMedianValue                       

#### Duplicates

In [150]:
aggregated[aggregated.duplicated(keep = False)]

,countryCode,monitoringSiteIdentifier,monitoringSiteIdentifierScheme,parameterWaterBodyCategory,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,procedureAnalysedMatrix,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,...,resultQualityMeanBelowLOQ,resultMeanValue,resultQualityMaximumBelowLOQ,resultMaximumValue,resultQualityMedianBelowLOQ,resultMedianValue,resultStandardDeviationValue,parameterSampleDepth,resultObservationStatus,UID


In [151]:
aggregated[['parameterSamplingPeriodStart', 'parameterSamplingPeriodEnd']] = aggregated['parameterSamplingPeriod'].str.split('--',
                                                                                                                            expand = True)

In [152]:
aggregated[aggregated['parameterSamplingPeriodStart'].isna()]

,countryCode,monitoringSiteIdentifier,monitoringSiteIdentifierScheme,parameterWaterBodyCategory,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,procedureAnalysedMatrix,resultUom,phenomenonTimeReferenceYear,parameterSamplingPeriod,...,resultQualityMaximumBelowLOQ,resultMaximumValue,resultQualityMedianBelowLOQ,resultMedianValue,resultStandardDeviationValue,parameterSampleDepth,resultObservationStatus,UID,parameterSamplingPeriodStart,parameterSamplingPeriodEnd
1854952,EE,EESJA9303000,eionetMonitoringSiteCode,RW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2013,2013,...,False,0.007,True,0.0050,NaN,-9999.0,NaN,10367820,NaN,NaN
1854954,EE,EESJA9303000,eionetMonitoringSiteCode,RW,CAS_14797-65-0,Nitrite,W,mg{NO2}/L,2013,2013,...,True,0.002,True,0.0020,NaN,-9999.0,NaN,10367822,NaN,NaN
1854955,EE,EESJA9303000,eionetMonitoringSiteCode,RW,CAS_14798-03-9,Ammonium,W,mg{NH4}/L,2013,2013,...,True,0.010,True,0.0100,NaN,-9999.0,NaN,10367823,NaN,NaN
1854956,EE,EESJA9303000,eionetMonitoringSiteCode,RW,CAS_7439-92-1,Lead and its compounds,W-DIS,ug/L,2013,2013,...,False,0.540,True,0.1000,NaN,-9999.0,NaN,10367824,NaN,NaN
1854957,EE,EESJA9303000,eionetMonitoringSiteCode,RW,CAS_7440-02-0,Nickel and its compounds,W-DIS,ug/L,2013,2013,...,False,2.300,False,0.2000,NaN,-9999.0,NaN,10367825,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4473134,DK,DK620014,eionetMonitoringSiteCode,RW,CAS_7723-14-0,Total phosphorus,W,mg{P}/L,2020,2020,...,False,0.210,False,0.0595,0.048317,0.0,NaN,17752178,NaN,NaN
4473135,DK,DK660014,euMonitoringSiteCode,RW,EEA_3161-02-2,Total oxidised nitrogen,W-DIS,mg{N}/L,2020,2020,...,False,9.800,False,2.0000,3.145656,0.0,NaN,17752179,NaN,NaN
4473136,DK,DK660014,euMonitoringSiteCode,RW,EEA_31615-01-7,Total nitrogen,W,mg{N}/L,2020,2020,...,False,12.000,False,2.2000,3.538933,0.0,NaN,17752180,NaN,NaN
4473137,DK,DK660014,euMonitoringSiteCode,RW,CAS_14265-44-2,Phosphate,W-DIS,mg{P}/L,2020,2020,...,False,0.200,False,0.1100,0.057543,0.0,NaN,17752181,NaN,NaN


In [153]:
aggregated['parameterSamplingPeriodStart'] = np.where(aggregated['parameterSamplingPeriodStart'].isna(),
                                                     aggregated['phenomenonTimeReferenceYear'],
                                                     aggregated['parameterSamplingPeriodStart'])

In [154]:
aggregated['parameterSamplingPeriodStart'].isna().sum()

0

In [155]:
aggregated['parameterSamplingPeriodEnd'] = np.where(aggregated['parameterSamplingPeriodEnd'].isna(),
                                                   aggregated['phenomenonTimeReferenceYear'],
                                                   aggregated['parameterSamplingPeriodEnd'])

In [156]:
aggregated['parameterSamplingPeriodEnd'].isna().sum()

0

In [157]:
aggregated = aggregated.drop(['parameterSamplingPeriod', 'resultObservationStatus', 'parameterSampleDepth'],
                            axis = 1)

#### Filter to only River and Lake waters (RW, LW)

In [158]:
aggregated_rw_lw = aggregated[(aggregated['parameterWaterBodyCategory'] == 'RW') |
                              (aggregated['parameterWaterBodyCategory'] == 'LW')]

In [159]:
aggregated_rw_lw

,countryCode,monitoringSiteIdentifier,monitoringSiteIdentifierScheme,parameterWaterBodyCategory,observedPropertyDeterminandCode,observedPropertyDeterminandLabel,procedureAnalysedMatrix,resultUom,phenomenonTimeReferenceYear,procedureLOQValue,...,resultQualityMeanBelowLOQ,resultMeanValue,resultQualityMaximumBelowLOQ,resultMaximumValue,resultQualityMedianBelowLOQ,resultMedianValue,resultStandardDeviationValue,UID,parameterSamplingPeriodStart,parameterSamplingPeriodEnd
0,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2004,NaN,...,NaN,0.001956,NaN,0.002608,NaN,0.001956,NaN,1,2004-01,2004-12
1,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2005,NaN,...,NaN,0.033000,NaN,0.052000,NaN,0.030000,0.016050,2,2005-01,2005-12
2,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2006,0.00163,...,False,0.014861,False,0.020294,False,0.015324,0.005802,3,2006-01,2006-12
3,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14265-44-2,Phosphate,W,mg{P}/L,2007,0.00163,...,False,0.014250,False,0.017118,False,0.013912,0.002409,4,2007-01,2007-12
4,AL,AL1,eionetMonitoringSiteCode,LW,CAS_14797-55-8,Nitrate,W,mg{NO3}/L,2005,NaN,...,NaN,0.442700,NaN,0.752590,NaN,0.442700,0.101800,5,2005-01,2005-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4550554,RO,RO85010,euMonitoringSiteCode,RW,CAS_7440-66-6,Zinc and its compounds,W-DIS,ug/L,2017,50.00000,...,True,50.000000,True,50.000000,True,50.000000,0.000000,17834956,2017-01-01,2017-12-31
4550555,RO,RO85010,euMonitoringSiteCode,RW,CAS_7440-47-3,Chromium and its compounds,W-DIS,ug/L,2017,1.00000,...,True,1.000000,True,1.000000,True,1.000000,0.000000,17834957,2017-01-01,2017-12-31
4550556,RO,RO85010,euMonitoringSiteCode,RW,CAS_7440-38-2,Arsenic and its compounds,W-DIS,ug/L,2017,0.10000,...,False,1.347500,False,2.920000,False,1.135000,1.153296,17834958,2017-01-01,2017-12-31
4550557,RO,RO85010,euMonitoringSiteCode,RW,EEA_31-02-7,Total suspended solids,W,mg/L,2017,10.00000,...,False,17.500000,False,26.000000,False,15.500000,6.137318,17834959,2017-01-01,2017-12-31


In [160]:
aggregated_rw_lw['parameterSamplingPeriodStart'] = pd.to_datetime(aggregated_rw_lw['parameterSamplingPeriodStart']) #, format="Y%/m%/d%"

C:\Users\elisa\AppData\Local\Temp\ipykernel_23448\3032630185.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_rw_lw['parameterSamplingPeriodStart'] = pd.to_datetime(aggregated_rw_lw['parameterSamplingPeriodStart']) #, format="Y%/m%/d%"


In [161]:
aggregated_rw_lw['parameterSamplingPeriodEnd'] = pd.to_datetime(aggregated_rw_lw['parameterSamplingPeriodEnd'])

C:\Users\elisa\AppData\Local\Temp\ipykernel_23448\3778761955.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aggregated_rw_lw['parameterSamplingPeriodEnd'] = pd.to_datetime(aggregated_rw_lw['parameterSamplingPeriodEnd'])


In [162]:
len(aggregated_rw_lw['UID'].unique()) # Primary key

3549404

In [163]:
aggregated_rw_lw.columns

Index(['countryCode', 'monitoringSiteIdentifier',
       'monitoringSiteIdentifierScheme', 'parameterWaterBodyCategory',
       'observedPropertyDeterminandCode', 'observedPropertyDeterminandLabel',
       'procedureAnalysedMatrix', 'resultUom', 'phenomenonTimeReferenceYear',
       'procedureLOQValue', 'resultNumberOfSamples',
       'resultQualityNumberOfSamplesBelowLOQ', 'resultQualityMinimumBelowLOQ',
       'resultMinimumValue', 'resultQualityMeanBelowLOQ', 'resultMeanValue',
       'resultQualityMaximumBelowLOQ', 'resultMaximumValue',
       'resultQualityMedianBelowLOQ', 'resultMedianValue',
       'resultStandardDeviationValue', 'UID', 'parameterSamplingPeriodStart',
       'parameterSamplingPeriodEnd'],
      dtype='object')

In [164]:
aggregated_rw_lw.shape

(3549404, 24)

In [165]:
aggregated_rw_lw = aggregated_rw_lw.fillna(0)

In [166]:
aggregated_rw_lw.isnull().sum()

countryCode                             0
monitoringSiteIdentifier                0
monitoringSiteIdentifierScheme          0
parameterWaterBodyCategory              0
observedPropertyDeterminandCode         0
observedPropertyDeterminandLabel        0
procedureAnalysedMatrix                 0
resultUom                               0
phenomenonTimeReferenceYear             0
procedureLOQValue                       0
resultNumberOfSamples                   0
resultQualityNumberOfSamplesBelowLOQ    0
resultQualityMinimumBelowLOQ            0
resultMinimumValue                      0
resultQualityMeanBelowLOQ               0
resultMeanValue                         0
resultQualityMaximumBelowLOQ            0
resultMaximumValue                      0
resultQualityMedianBelowLOQ             0
resultMedianValue                       0
resultStandardDeviationValue            0
UID                                     0
parameterSamplingPeriodStart            0
parameterSamplingPeriodEnd        

#### Save the cleaned dataset

In [167]:
aggregated.to_csv("Aggregated_cleaned.csv")

In [168]:
aggregated_rw_lw.to_csv("Aggregated_RW_LW_cleaned.csv")

<a id = 'SQL_connection'></a>
## 2. SQL Server connection
[Top](#top)

In [169]:
!pip install pyodbc

In [170]:
import pyodbc

In [172]:
# Connection working:
connection = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=DESKTOP-7D7TC79\SQLEXPRESS;'
                            'DATABASE=M7_Water_Quality_Analysis;'
                            'Trusted_Connection=yes;')

In [173]:
cursor = connection.cursor()

<a id = 'create_tables'></a>
### 2.1 Create tables in the SQL server database
[Top](#top)

<a id = 'table_emissions'></a>
#### 2.1.1 Create table with emissions water quantity data
[Top](#top)

In [102]:
cursor.execute("""
                DROP TABLE IF EXISTS emissions_data;
                
                CREATE TABLE emissions_data (
                    countryCode VARCHAR(10),
                    spatialUnitIdentifier VARCHAR(400),
                    spatialUnitIdentifierScheme VARCHAR(400),
                    observedPropertyDeterminandCode VARCHAR(400),
                    observedPropertyDeterminandLabel VARCHAR(400),
                    parameterEmissionsSourceCategory VARCHAR(10),
                    parameterEPRTRfacilities VARCHAR(10),
                    procedureEmissionsMethod VARCHAR(400),
                    resultObservationStatus VARCHAR(400),
                    Remarks VARCHAR(600),
                    UID INT PRIMARY KEY,
                    resultsEmissionsValueNEW FLOAT,
                    resultEmissionsUomNEW VARCHAR(10),
                    TimeReferenceStart INT,
                    TimeReferenceEnd INT)""")

for row in emissions_df_raw.itertuples():
    cursor.execute("""
                    INSERT INTO emissions_data (countryCode, spatialUnitIdentifier, spatialUnitIdentifierScheme,
                        observedPropertyDeterminandCode, observedPropertyDeterminandLabel, parameterEmissionsSourceCategory,
                        parameterEPRTRfacilities, procedureEmissionsMethod, resultObservationStatus, Remarks, UID,
                        resultsEmissionsValueNEW, resultEmissionsUomNEW, TimeReferenceStart, TimeReferenceEnd)
                    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                    row.countryCode,
                    row.spatialUnitIdentifier,
                    row.spatialUnitIdentifierScheme,
                    row.observedPropertyDeterminandCode,
                    row.observedPropertyDeterminandLabel,
                    row.parameterEmissionsSourceCategory,
                    row.parameterEPRTRfacilities,
                    row.procedureEmissionsMethod,
                    row.resultObservationStatus,
                    row.Remarks,
                    row.UID,
                    row.resultsEmissionsValueNEW,
                    row.resultEmissionsUomNEW,
                    row.TimeReferenceStart,
                    row.TimeReferenceEnd)
connection.commit()

In [175]:
pd.read_sql_query("""SELECT countryCode, observedPropertyDeterminandLabel, TimeReferenceStart
                     FROM emissions_data WHERE TimeReferenceStart = '2017'""", connection)

C:\Users\elisa\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,countryCode,observedPropertyDeterminandLabel,TimeReferenceStart
0,UK,Fluoranthene,2017
1,UK,"Total PAHs (4 PAHs: Benzo(a)pyrene, Benzo(b)fl...",2017
2,UK,Cyanazine,2017
3,UK,Diuron,2017
4,UK,Zinc and its compounds,2017
...,...,...,...
10097,NO,Total nitrogen,2017
10098,NO,Total nitrogen,2017
10099,NO,Total nitrogen,2017
10100,NO,Total nitrogen,2017


The connection to the SQL server database and the creation of the table were completed with success.

<a id = 'table_aggregated'></a>
#### 2.1.2 Create table with measured water quality data
[Top](#top)

In [94]:
cursor.execute("""
                DROP TABLE IF EXISTS measured_data;
                
                CREATE TABLE measured_data (
                    countryCode VARCHAR(10), 
                    monitoringSiteIdentifier VARCHAR(100),
                    monitoringSiteIdentifierScheme VARCHAR(100),
                    parameterWaterBodyCategory VARCHAR(10),
                    observedPropertyDeterminandCode VARCHAR(100),
                    observedPropertyDeterminandLabel VARCHAR(400),
                    procedureAnalysedMatrix VARCHAR(100),
                    resultUom VARCHAR(100),
                    phenomenonTimeReferenceYear INT,
                    procedureLOQValue FLOAT,
                    resultNumberOfSamples INT,
                    resultQualityNumberOfSamplesBelowLOQ INT,
                    resultQualityMinimumBelowLOQ VARCHAR(10),
                    resultMinimumValue FLOAT,
                    resultQualityMeanBelowLOQ VARCHAR(10),
                    resultMeanValue FLOAT,
                    resultQualityMaximumBelowLOQ VARCHAR(10),
                    resultMaximumValue FLOAT,
                    resultQualityMedianBelowLOQ VARCHAR(10),
                    resultMedianValue FLOAT,
                    resultStandardDeviationValue FLOAT,
                    UID INT PRIMARY KEY,
                    parameterSamplingPeriodStart DATE,
                    parameterSamplingPeriodEnd DATE)
                """)

for row in aggregated_rw_lw.itertuples():
    cursor.execute("""
                    INSERT INTO measured_data (countryCode, monitoringSiteIdentifier,
                  monitoringSiteIdentifierScheme, parameterWaterBodyCategory, observedPropertyDeterminandCode,
                  observedPropertyDeterminandLabel, procedureAnalysedMatrix,
                  resultUom, phenomenonTimeReferenceYear, procedureLOQValue, resultNumberOfSamples,
                  resultQualityNumberOfSamplesBelowLOQ, resultQualityMinimumBelowLOQ, resultMinimumValue,
                  resultQualityMeanBelowLOQ, resultMeanValue, resultQualityMaximumBelowLOQ, resultMaximumValue,
                  resultQualityMedianBelowLOQ, resultMedianValue, resultStandardDeviationValue,
                  UID, parameterSamplingPeriodStart, parameterSamplingPeriodEnd)
                    VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",
                  row.countryCode,
                  row.monitoringSiteIdentifier,
                  row.monitoringSiteIdentifierScheme,
                  row.parameterWaterBodyCategory,
                  row.observedPropertyDeterminandCode,
                  row.observedPropertyDeterminandLabel,
                  row.procedureAnalysedMatrix,
                  row.resultUom,
                  row.phenomenonTimeReferenceYear,
                  row.procedureLOQValue,
                  row.resultNumberOfSamples,
                  row.resultQualityNumberOfSamplesBelowLOQ,
                  row.resultQualityMinimumBelowLOQ,
                  row.resultMinimumValue,
                  row.resultQualityMeanBelowLOQ,
                  row.resultMeanValue,
                  row.resultQualityMaximumBelowLOQ,
                  row.resultMaximumValue,
                  row.resultQualityMedianBelowLOQ,
                  row.resultMedianValue,
                  row.resultStandardDeviationValue,
                  row.UID,
                  row.parameterSamplingPeriodStart,
                  row.parameterSamplingPeriodEnd)
connection.commit()

In [177]:
pd.read_sql_query("""SELECT observedPropertyDeterminandLabel, resultMeanValue FROM measured_data""", connection)

C:\Users\elisa\anaconda3\lib\site-packages\pandas\io\sql.py:762: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,observedPropertyDeterminandLabel,resultMeanValue
0,Phosphate,0.001956
1,Phosphate,0.033000
2,Phosphate,0.014861
3,Phosphate,0.014250
4,Nitrate,0.442700
...,...,...
3549399,Zinc and its compounds,50.000000
3549400,Chromium and its compounds,1.000000
3549401,Arsenic and its compounds,1.347500
3549402,Total suspended solids,17.500000


The connection to the SQL server database and the creation of the table were completed with success.